ChEn-3170: Computational Methods in Chemical Engineering Fall 2018 UMass Lowell; Prof. V. F. de Almeida **11Oct2018**

# 07. Computational Stoichiometry
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \DeclareMathOperator{\rank}{rank}
$

---
## Table of Contents
* [Introduction](#intro)
* [Stoichiometric matrix](#stoicmtrx)
* [Linear independent](#indepen)
* [Full-rank, sub-reaction mechanism](#subreact)
* [Reaction frequency](#rxnfreq)
---

## Introduction<a id="intro"></a>
"Stoichiometry is essentially the bookkeeping of the material components of a chemical system." Rutherford Aris in Elementary Chemical Reactor Analysis. Computational stoichiometry is the matrix analysis of the stoichiometric matrix of a chemical reaction mechanism. We will use it for two purposes: 
 + Evaluate sets of independent reactions
 + Evaluate reaction rates and species production rates

Recall [linear algebra notes.](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/ErfDAD_jL1tHkDrx29w89-MBcOgK4JAnEYSheRoxkL_0sw?e=fpzG5K)

[Notes](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/Evb2l8y2WNJCgNvJhcF0Pc4B-_TOOflJkiEAgCfICZwNVA?e=sV9YK0) on computational stoichiometry.

## Stoichiometric matrix<a id="stoicmtrx"></a>


In [ ]:
'''Open file for an ammonia oxidation reaction mechanism'''

# open file in reading mode 'r' (default), text 't' (default)
finput = open('data/ammonia-rxn.txt','rt')

!cat 'data/ammonia-rxn.txt'

In [ ]:
'''Build the reactions list'''

reactions = list()
for line in finput:
    reactions.append( line.strip() )
for r in reactions: 
    i = reactions.index(r)
    print('r%s'%i,': ',r)

In [ ]:
'''Shuffle the order of reactions'''

import random
random.shuffle( reactions )
for r in reactions: 
    i = reactions.index(r)
    print('r%s'%i,': ',r)

In [ ]:
'''Create the species list'''

species_tmp = list()  # temporary list for species
for r in reactions:
    left = r.split('<')
    right = r.split('>')
    species_left = left[0].split('+')
    species_right = right[1].split('+')
    species_rxn = species_left + species_right
    print('species_rxn =',species_rxn)
    for i in species_rxn:
        species_tmp.append( i.split(' ')[1] )    
print('\nspecies_tmp =',species_tmp)

species_filter = set(species_tmp) # filter species as a set

species = list( species_filter )  # convert species set to list 
print('\nspecies =\n',species)
print('# of species =',len(species))

In [ ]:
'''Create the stoichiometric matrix'''

import numpy as np
s_mtrx = np.zeros((len(reactions),len(species)))
for r in reactions:
    left = r.split('<')
    left_terms = left[0].split('+')
    for t in left_terms:
        coeff = float(t.split(' ')[0])
        i_row = reactions.index(r)
        species_member = t.split(' ')[1]
        j_col = species.index(species_member)
        s_mtrx[i_row,j_col] = -1.0 * coeff
        
    right = r.split('>')
    right_terms = right[1].split('+')
    for t in right_terms:
        coeff = float(t.split(' ')[0])
        i_row = reactions.index(r)
        species_member = t.split(' ')[1]
        j_col = species.index(species_member)
        s_mtrx[i_row,j_col] = 1.0 * coeff
        
print('species',species)
print('s_mtrx =\n',s_mtrx)
print('m x n =',s_mtrx.shape)

## Linearly independent reactions<a id="indepen"></a>


In [ ]:
'''How many reactions are independent?'''

from chen_3170.toolkit import lu_factorization

# using complete pivoting
(p_mtrx, q_mtrx, l_mtrx, u_mtrx, s_rank) = lu_factorization( s_mtrx, 'complete', pivot_tol=1e-8 )

print('my rank =',s_rank)
print('numpy rank = ',np.linalg.matrix_rank( s_mtrx, tol=1e-8 ))
np.set_printoptions(precision=2)
print('u_mtrx =\n',u_mtrx)

In [ ]:
'''How many reactions are independent?'''

# partial pivoting could fail
(p_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx, 'partial', pivot_tol=1e-8 )

print('my rank =',rank)
print('u_mtrx =\n',u_mtrx)

In [ ]:
'''How many reactions are independent? Let's break partial pivoting'''

# partial pivoting could fail; try all cases of reaction permutation
import math
import itertools
rxn_permutations = list( itertools.permutations(range(len(reactions))) )
print('# of permutations = ', len(rxn_permutations))
print('# of reactions!   = ',math.factorial(len(reactions)))

print(s_mtrx)
print(rxn_permutations[1200])
print(s_mtrx[rxn_permutations[1200],:])
#print(s_mtrx[[0,5,2,1,4,3,6],:])

for perm in rxn_permutations:
    (p_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx[perm,:], 'partial', pivot_tol=1e-8 )    
    assert rank == 3

print('done')

In [ ]:
'''How many reactions are independent?'''

# no pivoting will fail
(p_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx, pivot_tol=1e-8 )

## Full-rank, sub-reaction mechanisms<a id="subreact"></a>


In [ ]:
'''Total number of 3-reaction sets'''

# n choose k binomial formula
import math
from itertools import combinations
n_reactions = len(reactions)
reaction_sets = combinations(range(n_reactions),3)
print('3-reaction sets possible:',
      math.factorial(n_reactions)/math.factorial(n_reactions-s_rank)/math.factorial(s_rank))


In [ ]:
'''Finding sets of linearly independent reactions'''

sub_reactions = list()
for r in reaction_sets:
    s_mtrx_k = s_mtrx[r,:]
    (p_mtrx, q_mtrx, l_mtrx, u_mtrx, rank) = lu_factorization( s_mtrx_k, 'complete', pivot_tol=1e-8 )
#   rank = np.linalg.matrix_rank( s_mtrx_k, tol=1e-8 )
    if rank == s_rank:
        sub_reactions.append( (r, [reactions[i] for i in r]) )

print('# of sub_reactions =',len(sub_reactions))        
print(sub_reactions)

for s in sub_reactions:
    print('Linearly Independent Reaction Set %s'%sub_reactions.index(s))
    for (i,r) in zip(s[0],s[1]):
        print('r%s'%i,r)

## Reaction frequency<a id="rxnfreq"></a>
